<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Two: Feature Selection </h2>	


<h4> In this section, we would like you to select between 15 and 20 features to focus your model on. This will require significant explatoratory research. The first one is already implemented for you, and the next two are pre-specified.  </h4>

In [92]:
import pandas as pd
import numpy as np
import skimage.io as io

starting_data = pd.read_pickle("./starting_data.pkl")

In [94]:
# def get_intensity_based_features(image):
#     avg_channel = np.zeros(3)
#     for i in range(3):
#         avg_channel[i] = np.mean(image[:,:,i] )
#     a=np.argmax(image, axis=2)
#     ind, counts = np.unique(a, return_counts=True)
#     percent_max = counts/np.sum(counts)
#     return [avg_channel, percent_max]

# def getImageFeatures(image):
#     # handle condition when images are black-white
#     # by copying b/w image to all three channels
    
#     if len(image.shape) == 2:
#         temp = np.zeros((image.shape[0], image.shape[1], 3))
#         for i in range(3):
#             temp[:,:,i] = image
#         image = temp
    
#     m,n,k = image.shape
    
#     # Get full image features
#     full_image_features= get_intensity_based_features(image)
    
#     patch_min = int(m/4)
#     patch_max = int(3*m/4)
#     cropped = image[patch_min:patch_max, patch_min:patch_max, :]
    
#     # Get central cropped image features
#     cropped_image_features = get_intensity_based_features(cropped)
    
    
#     return [full_image_features, cropped_image_features] 

def normalize(image):
    if len(image.shape) == 2:
        temp = np.zeros((image.shape[0], image.shape[1], 3))
        for i in range(3):
            temp[:,:,i] = image
        image = temp
    return image

# Returns the average of the red-channel pictures for the images
def red_avg(image):
    return np.mean(image[:, :, 0])

# Returns the average of the green-channel pictures for the images
def blue_avg(image):
    return np.mean(image[:, :, 1])

# Returns the average of the blue-channel pictures for the images
def green_avg(image):
    return np.mean(image[:, :, 2])

def argmax(image):
    orig = np.zeros(3)
    a = np.argmax(image, axis = 2)
    vals, counts = np.unique(a, return_counts=True)
    for i in range(3):
        if i in vals:
            orig[i] = counts[np.where(vals == i)]
    percent_max = orig/np.sum(orig)
    if len(percent_max) != 3:
        print('Help')
    return percent_max

def crop(image):
    m, n, k = image.shape
    patch_min = int(m/4)
    patch_max = int(3*m/4)
    cropped = image[patch_min:patch_max, patch_min:patch_max, :]    
    return cropped

def contrast(image):
    vals = np.zeros(3)
    for i in range(3):
        vals[i] = (np.max(image[:, :, i]) - np.min(image[:, :, i]))
    return np.mean(vals)

def getImageFeatures(df):
    df["image"] = df["image"].apply(normalize)
    df["red"] = df["image"].apply(red_avg)
    df["green"] = df["image"].apply(green_avg)
    df["blue"] = df["image"].apply(blue_avg)
    df["argmax"] = df["image"].apply(argmax)
    df["contrast"] = df["image"].apply(contrast)
    
    df["cropped"] = df["image"].apply(crop)
    df["red_crop"] = df["cropped"].apply(red_avg)
    df["green_crop"] = df["cropped"].apply(green_avg)
    df["blue_crop"] = df["cropped"].apply(blue_avg)
    df["argmax_crop"] = df["cropped"].apply(argmax)
    df["contrast_crop"] = df["cropped"].apply(contrast)
    #Uncomment the following line when ready to make the final dataframe for Part 3!
    #df = df.drop(labels = ['image', 'filename', 'cropped'], axis = 1)
    return df

starting_data = getImageFeatures(starting_data)
starting_data.head()
#starting_data['argmax'].str.split(',', expand = True)

,image,class,filename,aspect,size,red,green,blue,argmax,contrast,cropped,red_crop,green_crop,blue_crop,argmax_crop,contrast_crop
0,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",0,airplanes_0001.jpg,2.426829,16384,0.719047,0.585263,0.693171,"[0.572326660156, 0.421020507813, 0.00665283203...",0.997897,"[[[0.817555625766, 0.868536017923, 0.833241900...",0.631888,0.537280,0.629310,"[0.536376953125, 0.46044921875, 0.003173828125]",0.982738
1,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",0,airplanes_0002.jpg,2.179348,16384,0.826595,0.646464,0.742442,"[0.971801757813, 0.0281982421875, 0.0]",0.979614,"[[[0.925732900582, 0.0671764897365, 0.02129049...",0.675802,0.410969,0.498426,"[0.984375, 0.015625, 0.0]",0.940746
2,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",0,airplanes_0003.jpg,2.381818,16384,0.666337,0.437090,0.578738,"[0.705322265625, 0.0567016601563, 0.237976074219]",0.993714,"[[[0.454641544118, 0.543550857843, 0.600560685...",0.611749,0.193338,0.418503,"[0.809326171875, 0.0654296875, 0.125244140625]",0.864494
3,"[[[0.678936887255, 0.741681985294, 0.729917279...",0,airplanes_0004.jpg,2.311765,16384,0.597665,0.306170,0.521160,"[0.83544921875, 0.161743164063, 0.0028076171875]",0.994251,"[[[0.427389705882, 0.389491421569, 0.241727941...",0.575981,0.148851,0.407758,"[0.94384765625, 0.052734375, 0.00341796875]",0.969045
4,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",0,airplanes_0005.jpg,2.244318,16384,0.576913,0.337259,0.589097,"[0.404968261719, 0.515441894531, 0.07958984375]",0.998115,"[[[0.259420955882, 0.340962009804, 0.051715686...",0.393732,0.183056,0.401980,"[0.2626953125, 0.493408203125, 0.243896484375]",0.994631


Define more features above, performing any EDA research below. We expect all external sources sited, and a couple significant different graphs indicating some form of EDA. 

<h4> Graphs </h4>

In [4]:
# store -r starting_data

<h4> Sources </h4>

In [5]:
files=[]
for i,image in enumerate(starting_data["image"]):
    if len(image.shape) < 3:
        files.append(starting_data.iloc[i, 2])
print(files)

['blimp_0022.jpg', 'comet_0006.jpg', 'comet_0011.jpg', 'comet_0013.jpg', 'comet_0021.jpg', 'comet_0036.jpg', 'comet_0038.jpg', 'comet_0041.jpg', 'comet_0049.jpg', 'comet_0052.jpg', 'comet_0053.jpg', 'comet_0057.jpg', 'comet_0058.jpg', 'crab_0045.jpg', 'dolphin_0025.jpg', 'gorilla_0128.jpg']


<h4> DataFrame Creation </h4>

In [6]:
def feature_frame(df):
    return df
    #Returns data-frame with all the features now inside, and calculated


In [ ]:
feature_frame(data_from_nb1)